The optimization technique that was tried in this experiment was doing ginni impurity calculation instead of entropy calculation. 

The accuracy result obtained from both entropy and ginni dont differ by much, but ginni is slightly easier to calculate as it doesnot have the log function. 

Also, ginni impurity is calculated faster as compared to  entropy because of the absence of log function

When the tree was created using the entropy impurity(initial entropy), the accuracy obtained was around 82( can be seen in the second ipynb file) but when tried using ginni impurity, the average was slightly lower(81.912(commented code here)) because of all the cross-validation samples, one gave bad accuracy. All the others were almost the same and even some where greater. One example of that is shown in this code(83.67) compared to 82.65 in entropy (shown in first ipynb).

The major advantage of ginni over entropy is the fact that it is computationally fast. 


In [1]:
# CS6510 HW 1 Code Skeleton
# Please use this outline to implement your decision tree. You can add any code around this.
import csv
import math
import numpy as np
import time
import random
# Enter You Name Here
myname = "Sharath" # or "Doe-Jane-"

In [2]:
# Function that calculates the initialGinni before the split occcurs by converting it into numpy array and calling
# the ginni function that calcuates the ginni value for a set of rows
def calculateInitialGinni(training_set):
    training_set=np.asarray(training_set).astype("float")
    initial_ginni = ginni(training_set)
    return initial_ginni

In [3]:
#Function to calculate the InformationGain using the ginni values
def calculateIG(parentGinni, currentGinni):
    return (parentGinni-currentGinni)

In [4]:
#Alternate method when to calculate ginni when there are just two classes
'''
   def ginniForm(class0count, class1count):
    if(class0count>0 and class1count>0):
        probclass0= float(class0count/(class0count+class1count))
        probclass1=1-probclass0
    else:
        probclass0=0
        probclass1=0
        return 0
    ginniCal= 1-((probclass0**2)+(probclass1**2))
    return ginniCal
'''

'\n   def ginniForm(class0count, class1count):\n    if(class0count>0 and class1count>0):\n        probclass0= float(class0count/(class0count+class1count))\n        probclass1=1-probclass0\n    else:\n        probclass0=0\n        probclass1=0\n        return 0\n    ginniCal= 1-((probclass0**2)+(probclass1**2))\n    return ginniCal\n'

In [5]:
#Alternate function for getting ginni for a particular column when without using numpy
'''
def ginniVal(training_set,column_index,value):
    classg=[row for row in training_set if float(row[column_index])>=value]
    classl=[row for row in training_set if float(row[column_index])< value]
    count_classg= len(classg)
    count_classl=len(classl)
    if(count_classg==0 and count_classl==0):
        prob_classg=0
        prob_classl=0
    else:
        prob_classg= float(count_classg/(count_classg+ count_classl))
        prob_classl= 1-prob_classg
        
    classg0= [row for row in classg if float(row[-1])==0]
    count_classg0=len(classg0)
    classl0= [row for row in classl if float(row[-1])==0]
    count_classl0=len(classl0)
    count_classg1= len(classg)-count_classg0
    count_classl1=len(classl)-count_classl0
    
    entropyg= ginniForm(count_classg0, count_classg1)
    entropyl= ginniForm(count_classl0, count_classl1)
    
    giniIndex= prob_classg*entropyg+ prob_classl* entropyl
    
    #giniIndex= prob_classg*(1-((prob_g0*prob_g0)+(prob_g1*prob_g1)))+ prob_classl*(1-((prob_l0*prob_l0)+(prob_l1*prob_l1)))
    return giniIndex
  '''

'\ndef ginniVal(training_set,column_index,value):\n    classg=[row for row in training_set if float(row[column_index])>=value]\n    classl=[row for row in training_set if float(row[column_index])< value]\n    count_classg= len(classg)\n    count_classl=len(classl)\n    if(count_classg==0 and count_classl==0):\n        prob_classg=0\n        prob_classl=0\n    else:\n        prob_classg= float(count_classg/(count_classg+ count_classl))\n        prob_classl= 1-prob_classg\n        \n    classg0= [row for row in classg if float(row[-1])==0]\n    count_classg0=len(classg0)\n    classl0= [row for row in classl if float(row[-1])==0]\n    count_classl0=len(classl0)\n    count_classg1= len(classg)-count_classg0\n    count_classl1=len(classl)-count_classl0\n    \n    entropyg= ginniForm(count_classg0, count_classg1)\n    entropyl= ginniForm(count_classl0, count_classl1)\n    \n    giniIndex= prob_classg*entropyg+ prob_classl* entropyl\n    \n    #giniIndex= prob_classg*(1-((prob_g0*prob_g0)+(pr

In [6]:
# return the ginni value of a partiucular set of rows
# does it by splitting the rows according to the labels and obtain the probability of the labels and using the formula
#--- 1-summation(p^2)
def ginni(training_set):
    if(training_set.shape[0]==0):
        return 0
    training_set1=training_set[:,-1]
    unique_elements, counts_elements = np.unique(training_set1, return_counts=True) # get the count and type of label in the rows
    dic=dict(zip(unique_elements,counts_elements)) # create a dictionary with the label and count 
    p = []
    for label in dic:
        p.append(dic[label]/float(training_set1.shape[0])) # get the probability of every label into an array so that it is easy for numpy calculation
    return 1-np.sum(np.multiply(p,p))

In [7]:
# calculate ginni instead of entropy using the formula: (1-summation(p^2)) and returns the 
#ginni obtained for a particular column
def ginniVal(training_set, column_index, value):
    training_set_left = [row for row in training_set if float(row[column_index]) >= value]
    training_set_right = [row for row in training_set if float(row[column_index])< value]
    training_set_left = np.asarray(training_set_left,dtype='float')
    training_set_right = np.asarray(training_set_right,dtype='float')
    p =float((training_set_left.shape[0])/(training_set_left.shape[0]+training_set_right.shape[0]))
    return p*ginni(training_set_left)+(1-p)*ginni(training_set_right)

In [8]:
# Find the best split that is possible for the given training_set by calculating ginni and information gain
# then return the column that has the maximum information gain of the other rows
def findSplit(training_set):
    local_ginni=0
    local_column=None
    best_ginni=1
    best_column=None
    col_mean_final=0
    bestIg=0
    parentGinni=calculateInitialGinni(training_set)

    for col in range(0,len(training_set[0])-1):
        
        col_data= [float(row[col]) for row in training_set]
        col_mean=float(np.sum(col_data)/len(col_data)) # get col_mean for the remaining rows
        
        local_ginni= ginniVal(training_set,col,col_mean)
        localIg= calculateIG(parentGinni,local_ginni)
        
        if localIg>= bestIg:
            bestIg=localIg
            best_ginni=local_ginni
            best_column = col
            col_mean_final=col_mean
    return  best_ginni,best_column,col_mean_final

In [9]:
#Partition the rows into the rows that follow the condition and others that dont for a column based on a value
def partition(training_set, column_index, value):
    true_rows= [row for row in training_set if float(row[column_index]) >= value]
    false_rows = [row for row in training_set if float(row[column_index])< value]
    return true_rows, false_rows

In [10]:
# get the class of data in case it cant be split further i.e all the data in this particular class belong to
# the same class. So we just take the first label and set it as the output label for the decision tree node
def getClass(training_set, column_index,value):
    true_label= None
    false_label=None
    true_rows= [row for row in training_set if float(row[column_index]) >= value]
    false_rows = [row for row in training_set if float(row[column_index])< value]
    
    if len(true_rows)>0:
        true_label = true_rows[0][-1]
    if len(false_rows)>0:
        false_label= false_rows[0][-1]
        
    if false_label=="1":
        false_label="class1"
    if true_label=="0":
        true_label="class0"
    if false_label=="0":
        false_label="class0"
    if true_label=="1":
        true_label="class1"
    if len(true_rows)> len(false_rows):
        return true_label
    else:
        return false_label
#     return true_label, false_label

In [11]:
class DecisionTree():
    tree = {}

    def learn(self, training_set,index):
        #implement this function
        value=0
        true_index=0
        false_index=0
        gini,column_name,value = findSplit(training_set)
    
        #check terminating criteria with gini= 0 which means it is evenly being split
        if gini==0:
            return getClass(training_set, column_name, value)
             
        # get the true and false rows based on the criteria
        true_rows, false_rows = partition(training_set,column_name,value)
        
        # Make the recursive call for both true and false rows and get the index
        true_index = self.learn(true_rows, 2*index)
        
        false_index= self.learn(false_rows,2*index+1)
        
        #use the index the insert the data required into the tree
       
        self.tree[index]=[column_name,value,{"true_rows":true_index,"false_rows":false_index}]
        return index
    

    def classify(self, test_instance):
        result = 0 # baseline: always classifies as 0
        index=1 # root index
        while 1:
            value= self.tree[index]
            column_num= value[0]
            values=value[1]
            if float(test_instance[column_num])>=float(values):
                result= value[2]["true_rows"] # the third item in the tree is a dictionary that contains both the T and F ids
            else:
                result=value[2]["false_rows"]
            if result=="class1" or result=="class0":
                break
            else:
                index=result
        if result=="class1":
            result="1"
        else:
            result="0"
        return result


In [12]:
def run_decision_tree():

    # Load data set
    with open("assign1-q1-data.csv") as f:
        next(f, None)
        data = [tuple(line) for line in csv.reader(f, delimiter=",")]

    print("Number of records: %d" % len(data))
    sumOfaccuracy=0
    # Split training/test sets
    # You need to modify the following code for cross validation.
    #######Comment it for average case testing###################
    K = 10
    training_set = [x for i, x in enumerate(data) if i % K != 7]
    test_set = [x for i, x in enumerate(data) if i % K ==7]
    columns_set={}
    tree = DecisionTree()
    # Construct a tree using training set
    tree.learn( training_set,1)

    # Classify the test set using the tree we just constructed
    results = []
    for instance in test_set:
        result = tree.classify( instance[:-1] ) # prints all values except the last one
        results.append( result == instance[-1])

    # Accuracy
    accuracy = float(results.count(True))/float(len(results))
    print("accuracy: %.4f" % accuracy)
    ###############till-here######################
    
    #####Un-comment this for average of ginni impurity over 10 sets###########
#     K = 10
#     sumOfaccuracy=0
#     for j in range(0,10):
#         training_set = [x for i, x in enumerate(data) if i % K != j]
#         test_set = [x for i, x in enumerate(data) if i % K ==j]
#         columns_set={}
#         tree = DecisionTree()
#         # Construct a tree using training set
#         tree.learn( training_set,1)

#         # Classify the test set using the tree we just constructed
#         results = []
#         for instance in test_set:
#             result = tree.classify( instance[:-1] ) # prints all values except the last one
#             results.append( result == instance[-1])
#         # Accuracy
#         accuracy = float(results.count(True))/float(len(results))
#         sumOfaccuracy = sumOfaccuracy+accuracy
#         print("accuracy: %.4f" % accuracy)
#     print("average-accracy for 10 splits is", float(sumOfaccuracy/10))
    ###########till-here######################################
    
    # Writing results to a file (DO NOT CHANGE)
    f = open(myname+"result.txt", "w")
    f.write("accuracy: %.4f" % accuracy)
    f.close()

In [13]:
if __name__ == "__main__":
    run_decision_tree()

Number of records: 4898
accuracy: 0.8367
